In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def canny(image):
    gray = cv2.cvtColor(lane_image, cv2.COLOR_RGB2GRAY) #image change color image to gray scale image. bcz, it is easy to read image.
    blur = cv2.GaussianBlur(gray, (5,5), 0) # blur use to the image create blur and match the color with nearer pixel scale.
    canny = cv2.Canny(blur, 50, 150) #canny use for the image all pixel b/w the 50 and 150. so, edges in 150 and other pixel 50.
    return canny
    
    
image = cv2.imread('Road_image.jpeg') # imread use for the read image
lane_image = np.copy(image) #when we change in lane_image that time also change in original image
canny = canny(lane_image)  #canny function call

plt.imshow(canny) # imshow use for the show image with result name screen
plt.show()
#cv2.waitkey(0)

after the graph plot we are consider only requre part of road with help of x and y graph coordinets. part look like a tringle on x axis 200 to 1100. and on y axis 0 to 250.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def make_coordinates(image, line_parameters):
    slope, intercept = line_parameters
    y1 = image.shape[0]
    y2 = int(y1*(3/5))
    x1 = int((y1 - intercept)/slope)
    x2 = int((y2 - intercept)/slope)
    return np.array([x1, y1, x2, y2])

def average_slope_intercept(image, lines):
    left_fit = []
    right_fit = []
    for line in lines:
        x1, y1, x2, y2 = line.reshape(4)
        parameters = np.polyfit((x1, x2), (y1, y2), 1)
        print(parameters)
        slope = parameters[0]
        intercept = parameters[1] #left side line have negative slope and right side line have positive slope.
        if slope < 0:
            left_fit.append((slope, intercept))
        else:
            right_fit.append((slope, intercept))
    left_fit_average = np.average(left_fit, axis=0)
    right_fit_average = np.average(left_fit, axis=0)
    left_line = make_coordinates(image, left_fit_average)
    right_line = make_coordinates(image, right_fit_average)
    #print(left_fit_average, 'left')
    #print(right_fit_average,'right')
    return np.array([left_line, right_line])
            
def canny(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) #image change color image to gray scale image. bcz, it is easy to read image.
    blur = cv2.GaussianBlur(gray, (5,5), 0) # blur use to the image create blur and match the color with nearer pixel scale.
    canny = cv2.Canny(blur, 50, 150) #canny use for the image all pixel b/w the 50 and 150. so, edges in 150 and other pixel 50.
    return canny

def display_lines(image, Lines): 
    line_image = np.zeros_like(image)
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line.reshape(4)
            cv2.line(line_image, (x1, y1), (x2, y2), (255, 0, 0), 10)
    return line_image

def region_of_interest(image):
    height = image.shape[0]
    polygons = np.array([
    [(430,height), (1020, height), (800, 350)]
    ])
    mask = np.zeros_like(image) # same triangle is draw on the black (zeros means black color) backgraund image. same amaount of pixel and size.
    cv2.fillPoly(mask, polygons, 255) #fillpoly use to fill the tringle  with 255 intensity (means white triangle)
    masked_image = cv2.bitwise_and(image, mask)#Computing the bitwise & of both images as we saw earlier in the theory section, takes the bitwise & of each homologous pixel both arrays, ultimately masking the canny image to only show the region of interest traced by the polygonal contour of the mask.
    return masked_image

image = cv2.imread('Road_image.jpeg') # imread use for the read image
lane_image = np.copy(image) #when we change in lane_image that time also change in original image
canny_image = canny(lane_image)  #canny function call
cropped_image = region_of_interest(canny_image)
lines = cv2.HoughLinesP(cropped_image, 2, np.pi/180, 100,np.array([]), minLineLength = 40, maxLineGap = 5)
averaged_lines = average_slope_intercept(lane_image, lines)
line_image = display_lines(lane_image, averaged_lines)
combo_image = cv2.addWeighted(lane_image, 0.8, line_image, 1, 1)
plt.imshow(cropped_image) # imshow use for the show image with result name screen
#cv2.imshow("result", cropped_image)
#cv2.waitkey(0)

In [ ]:
gray = cv2.cvtColor(lane_image, cv2.COLOR_RGB2GRAY) #image change color image to gray scale image. bcz, it is easy to read image.
blur = cv2.GaussianBlur(gray, (5,5), 0) # blur use to the image create blur and match the color with nearer pixel scale.
canny = cv2.Canny(blur, 50, 150) #canny use for the image all pixel b/w the 50 and 150. so, edges in 150 and other pixel 50.

In [ ]:
plt.imshow(image)

In [ ]:
plt.imshow(lane_image)

In [ ]:
plt.imshow(gray)

In [ ]:
plt.imshow(blur)

In [ ]:
plt.imshow(canny)

In [ ]:
plt.imshow(cropped_image)

In [ ]:
plt.imshow(line_image)

In [ ]:
plt.imshow(combo_image)

In [ ]:
cap = cv2.VideoCapture("Road_video.mp4")
while(cap.isOpened()):
    _, frame = cap.read()
    canny_image = canny[frame]
    cropped_canny = region_of_interest(canny_image)
    lines = cv2.HoughLinesP(cropped_canny, 2, np.pi/180, 100, np.array([]), minLineLength=40, maxLineGap=5)
    averaged_lines = average_slope_intercept(frame, lines)
    line_image = display_lines(frame, averaged_lines)
    combo_image = cv2.addWeighted(frame, 0.8, line_image, 1, 1)
    cv2.imshow("result", combo_image)
    if cv2.waitKey(1) == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()